# Sentiment Analysis in the Browser

In this notebook, we will show how to create a `.air` file to perform sentiment analysis in the browser using a neural network.  To do this, we will utilize the IMDB Movie Reviews dataset to build the initial model, prune the model using the `mann` package, and then package the model using the `aisquared` Python SDK.

## Dependencies

For this notebook, the following dependencies are required:

- `mann`
- `aisquared`

Both of these are available on [pypi](https://pypi.org) via `pip`.  The following cell also runs the commands to install these dependencies as well as imports them into the notebook environment, along with TensorFlow (which is a dependency of the `mann` package).

In [ ]:
! pip install mann
! pip install aisquared

from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import aisquared
import mann

## Model Creation

Now that the required packages have been installed and imported, it is time to create the sentiment analysis model.  To do this, we have to first download and preprocess the data, create the model, prune the model so that it can perform well in the browser, and then package the model in the `.air` format.  The following cells will go through an in-depth explanation of each of the steps in this process.

In [ ]:
# Loading the data

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(
    num_words = 10000,
    skip_top = 0,
    start_char = 1,
    oov_char = 2,
    index_from = 3
)
x_train = tf.keras.preprocessing.sequence.pad_sequences(
    x_train,
    maxlen = 512,
    padding = 'post',
    truncating = 'post'
)
x_test = tf.keras.preprocessing.sequence.pad_sequences(
    x_test,
    maxlen = 512,
    padding = 'post',
    truncating = 'post'
)

# Get the vocabulary
vocab = tf.keras.datasets.imdb.get_word_index()

# Add 2 to each vocab value to ensure matching with the needed values
vocab = {
    k : v + 2 for k, v in vocab.items()
}


In [ ]:
# Create the model

input_layer = tf.keras.layers.Input(x_train.shape[1:])
embedding_layer = tf.keras.layers.Embedding(
    10000,
    4
)(input_layer)
x = tf.keras.layers.Flatten()(embedding_layer)
for _ in range(5):
    x = mann.layers.MaskedDense(1000, activation = 'relu')(x)
output_layer = mann.layers.MaskedDense(1, activation = 'sigmoid')(x)

model = tf.keras.models.Model(input_layer, output_layer)
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

In [ ]:
# Prune the model and train it
model = mann.utils.mask_model(
    model,
    50,
    x = x_train[:1000],
    y = y_train[:1000].reshape(-1, 1)
)
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

# Create a pruning callback that will increase pruning rate as performance improves
callback = mann.utils.ActiveSparsification(
    performance_cutoff = 0.8,
    starting_sparsification = 50
)

# Train the model with the sparsification callback
model.fit(
    x_train,
    y_train.reshape(-1,1),
    epochs = 1000,
    batch_size = 512,
    validation_split = 0.2,
    verbose = 2,
    callbacks = [callback]
)

# Now that the model has been trained, convert all model layers to built-in TensorFlow layers
model = mann.utils.remove_layer_masks(model)

In [ ]:
# Check model performance
preds = (model.predict(x_test) >= 0.5).astype(int)
print('Model Performance on Test Data:')
print('\n')
print(confusion_matrix(y_test, preds))
print(classification_report(y_test, preds))

# Save the model
model.save('SentimentClassifier.h5')

## Package the Model

Now that the model has been created, we can package the model into a single `.air` file that enables integration into the browser.

To perform this packaging, we will be utilizing the `aisquared` package `DocumentPredictor` class.

In [ ]:
aisquared.base.DocumentPredictor(
    model_path = 'SentimentClassifier.h5',
    vocabulary = vocab,
    sequence_length = 512,
    name = 'SentimentClassifier',
    label_map = ['Negative', 'Positive', 'Inconclusive'],
    include_probability = True,
    remove_punctuation = False,
    max_vocab = 9999
).compile()